# Introduction du problème

Dans cet exercice, je veux montrer comment utiliser les techniques d'apprentissage automatique pour étudier la similarité entre certaines des différentes pages de Wikipedia. Plus précisément, je souhaiterais estimer la similarité des pages concernant des personnages publiques americains. 

La base de données a été fournie par l'Université de Washington dans le cadre d'un de leurs cours universitaires. J'utiliserai principalement le package GraphLab qui a été developpé par une partie des membres de l'équipe de Statistique et Apprentissage Automatique de l'Université de Washington - Seattle.

Pour accomplir cette tâche, j'emploierai l'algorithme TF-IDF (Term Frequency - Inverse Document Frequency). L'idée fondamentale de l'algorithme est de compter le nombre total de fois où chaque mot apparaît dans l'article, en obtenant un vecteur. On peut alors donner une estimation de proximité des deux différents articles. Par le produit scalaire de deux différents vecteurs, on obtient en fait un nombre qui peut être utilisé pour déterminer la proximité des deux pages différentes. Une telle méthode a deux problèmes fondamentaux: premièrement, le vecteur dépend de la longueur de l'article; deuxièment, les mots très communs (qu'en langue français pourraient être le mot 'le', ou 'de', etc.) seraient d'une grande importance. 

Il faut donc normaliser le vecteur de deux manières différentes, d'une part en considérant sa longueur, et d'aute part en considérant la quantité de mots communs présente. Pour la première normalisation, il est suffisant de choisir une norme vectorielle et de normaliser le vecteur par sa norme associée. Pour le deuxième cas, on veut que les mots plus communs deviennent moins importants, idéalement on voudrait que le poids associé soit 0. Mieux encore, on voudrait sélectionner des mots qui soient relativement courants dans l'article choisi, mais très rares dans la totalité des articles selectionnés. 

On appelle alors w le mot qui nous intéresse, N le numéro total de documents et N(w) le nombre de documents qui contient le mot w. Étant donné un article et un mot w y présent, on multiplie w par f(w) = log (N/1+N(w)). On peut remarquer que, pour N suffisament grand, un mot commun à presque tous les articles aura un poids proche de 0, exactement ce que l'on veut obtenir.

Le package GraphLab a une fonction qui s'appelle tf_idf et qui calcule l'algorithme TF-IDF sans qu'il y ait besoin de normaliser le vecteur. Je vais présenter cette fonction plus en détail pr la suite.



# Les donnés

Les données sont un ensemble de biographies de personnes connues, biographies extraites de la version anglaise de Wikipedia. 

On va maintenant initialiser le package GraphLab.

In [1]:
import graphlab

[WARNING] Unable to write current GraphLab Create license to /home/roberto/.graphlab/config. Ensure that this user account has write permission to /home/roberto/.graphlab/config to save the license for offline use.
[INFO] 1446672366 : INFO:     (reap_unused_temp_files:277): Deleting orphaned temp directory found in /var/tmp/graphlab-roberto/10719
1446672366 : INFO:     (reap_unused_temp_files:277): Deleting orphaned temp directory found in /var/tmp/graphlab-roberto/6589
1446672366 : WARNING:  (delete_proc_directory:243): Unable to delete "/var/tmp/graphlab-roberto/6589/pickle13809fc6-b94c-4075-be44-a5d70a148fcc"
1446672366 : INFO:     (reap_unused_temp_files:277): Deleting orphaned temp directory found in /var/tmp/graphlab-roberto/10721
1446672366 : INFO:     (reap_unused_temp_files:277): Deleting orphaned temp directory found in /var/tmp/graphlab-roberto/10722
1446672366 : INFO:     (reap_unused_temp_files:277): Deleting orphaned temp directory found in /var/tmp/graphlab-roberto/10720

In [2]:
pages = graphlab.SFrame('/home/roberto/Desktop/Python/people_wiki.gl')


On regarde le type des donnés qu'on obtient.

In [3]:
pages.head()


URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


On peut remarquer que chaque ligne a trois données différentes: l'Url de la page, le titre de la page (dans ce cas, le nom du personnage) et le texte comme suite de mots, sans aucun formatage particulier. Les données sont déjà en très bon état et prêtes pour être utilisées. Comme contrôle ultérieur, je regarde les dernières dix pages.

In [4]:
pages.tail()

URI,name,text
<http://dbpedia.org/resource/Rod_Wilt> ...,Rod Wilt,rod wilt is a formerrepublican member of the ...
<http://dbpedia.org/resource/Scott_Baker_(judge)> ...,Scott Baker (judge),sir thomas scottgillespie baker born 10 ...
<http://dbpedia.org/resource/Dragoljub_Ojdani% ...,Dragoljub Ojdani%C4%87,dragoljub ojdani serbiancyrillic born june 1 ...
<http://dbpedia.org/resource/Oz_Bengur> ...,Oz Bengur,osman oz bengur bornfebruary 23 1949 is an ...
<http://dbpedia.org/resource/Dee_Brown_(basket ...,"Dee Brown (basketball,born 1968) ...",decovan kadell dee brownborn november 29 1968 is ...
<http://dbpedia.org/resource/Olari_Elts> ...,Olari Elts,olari elts born april 271971 in tallinn estonia ...
<http://dbpedia.org/resource/Scott_F._Crago> ...,Scott F. Crago,scott francis crago bornjuly 26 1963 twin bro ...
<http://dbpedia.org/resource/David_Cass_(footb ...,David Cass (footballer),david william royce cassborn 27 march 1962 in ...
<http://dbpedia.org/resource/Keith_Elias> ...,Keith Elias,keith hector elias bornfebruary 3 1972 in lacey ...
<http://dbpedia.org/resource/Fawaz_Damrah> ...,Fawaz Damrah,fawaz mohammed damraharabic fawwz damra was ...


On remarque qu'il peut y avoir des problèmes avec certains caractères pour certains noms, comme par exemple pour Dragoljub Ojdani. On va alors regarder le corps de son article, et celui de Scott Baker comme contrôle, pour voir s'il y a des problèmes dans les caractères.

In [5]:
print pages[pages['name'] == 'Dragoljub Ojdani%C4%87']['text']

['dragoljub ojdani serbian cyrillic born june 1 1941 in ravni near uice kingdom of yugoslavia was former chief of the general staff and defence minister of yugoslavia he was convicted of deportation and forcible transfers by the ictyin 1958 he studied at the yugoslav military academy and graduated in 1964 he was deputy commander of the 37th corps with command in uice he was promoted to major general on 20 april 1992 and he became the commander of uice korpus under his command the uice corps was deployed in military operations in eastern bosnia during the war in bosnia and herzegovina he served as chief of the general staff first army of fry in 1993 and 1994 between 19941996 he was commander of the first army in 1996 he became deputy commander chief of the general staff in 1998 slobodan miloevi placed ojdani as a chief of the general staff of the yugoslav army he was also a chief of general staff during natos operation allied forcein february 2000 after the death of defence minister pav

In [6]:
print pages[pages['name'] == 'Scott Baker (judge)']['text']

['sir thomas scott gillespie baker born 10 december 1937 is a retired english court of appeal judgebaker is the eldest son of sir george baker a former high court judge who was president of the family division from 1971 to 1979 one of his brothers his honour judge baker qc was the resident judge at st albans crown courtbaker was educated at haileybury and imperial service college and studied at brasenose college oxford he was a member of chorleywood urban district council from 1964 to 1967 he married margaret joy baker on 10 february 1973 they had 2 sons and one 1 daughter together he was called to the bar at the middle temple in 1961 and practised in a range of legal areas including family finance cases and professional negligencehe became a recorder in 1976 and was appointed a queens counsel in 1978 he became a bencher at middle temple in 1985 he was a member of the committee that inquired into human fertilisation in 1982 to 1984 chaired by mary warnock which led to the human fertili

Apparemment il n'y a pas des problèmes majeurs dans les articles. Nous nous voulons connaître le nombre totale de pages.

In [7]:
print len(pages)

59071


On a 59071 différents personnages! Nous pouvons selectionner ceux qui nous préféront. On va plutôt se concentrer sur des personnages américains. Pour simplifier la notation, je vais définir une fonction qui retourne le texte de la page associé à un certain nom.

In [8]:
def text(string):
    return pages[pages['name'] == string]['text']

On a dit dans l'introduction que le nombre de mots est un élément important. On définit alors un nouveau élément de notre SFrame, le nombre total de mot. Le package graphlab possède une fonction specifique pour la tâche, text_analytics.count_words.

In [10]:
pages['word_count'] = graphlab.text_analytics.count_words(pages['text'])

In [15]:
pages[pages['name'] == 'Barack Obama']['word_count']

dtype: dict
Rows: ?
[{'operations': 1, 'represent': 1, 'office': 2, 'unemployment': 1, 'doddfrank': 1, 'over': 1, 'unconstitutional': 1, 'domestic': 2, 'major': 1, 'years': 1, 'against': 1, 'proposition': 1, 'seats': 1, 'graduate': 1, 'debate': 1, 'before': 1, 'death': 1, '20': 2, 'taxpayer': 1, 'representing': 1, 'obamacare': 1, 'barack': 1, 'to': 14, '4': 1, 'policy': 2, '8': 1, 'he': 7, '2011': 3, '2010': 2, '2013': 1, '2012': 1, 'bin': 1, 'then': 1, 'his': 11, 'march': 1, 'gains': 1, 'cuba': 1, 'school': 3, '1992': 1, 'new': 1, 'not': 1, 'during': 2, 'ending': 1, 'continued': 1, 'presidential': 2, 'states': 3, 'husen': 1, 'osama': 1, 'californias': 1, 'equality': 1, 'prize': 1, 'lost': 1, 'made': 1, 'inaugurated': 1, 'january': 3, 'university': 2, 'rights': 1, 'july': 1, 'gun': 1, 'stimulus': 1, 'rodham': 1, 'troop': 1, 'withdrawal': 1, 'brk': 1, 'nine': 1, 'where': 1, 'referred': 1, 'affordable': 1, 'attorney': 1, 'on': 2, 'often': 1, 'senate': 3, 'regained': 1, 'national': 2, 'cr

En utilisant la fonction 'text_analytics.tf_idf' de graphlab on calcule le TF-IDF pour chaque entrée de notre base de données et on ajoute une nouvelle columne avec cette valeur. On va définir aussi une fonction qui, en ayant comme input deux noms, donne leur distance (en utilisant la distance cosine). Plus la distance est petite, plus les deux personnes sont proches.


In [19]:
tfidf = graphlab.text_analytics.tf_idf(pages['word_count'])


In [20]:
pages['tfidf'] = tfidf['docs']

In [22]:
print pages[pages['name'] == 'Barack Obama']['tfidf']

[{'operations': 3.811771079388818, 'represent': 4.184100625900883, 'office': 5.2481728232196465, 'unemployment': 6.642689967371511, 'doddfrank': 9.887883100557085, 'over': 1.4878231559557336, 'unconstitutional': 6.8276123058655225, 'domestic': 8.967410686619141, 'major': 2.0581201293715634, 'years': 1.0752380994247055, 'against': 2.0079609791418744, 'proposition': 6.926052378678775, 'seats': 5.000043383940756, 'graduate': 2.6750971107885535, 'debate': 4.783959872037272, 'before': 1.4967823726683713, 'death': 3.1568650000750016, '20': 4.88376320446593, 'taxpayer': 7.431147327735781, 'representing': 3.535253704237518, 'obamacare': 9.04058524016988, 'barack': 5.067601534952048, 'to': 0.6572291275451891, '4': 2.437803530749586, 'policy': 6.095386282141427, '8': 2.7572509724892824, 'he': 1.493579903611068, '2011': 5.107041270312876, '2010': 3.185667920243947, '2013': 1.9545642372230505, '2012': 1.7938099524877322, 'bin': 5.6158573610975315, 'then': 1.4309354361561304, 'his': 2.8887260073502

Nous définons une nouvelle fonction qui, étant donnés deux différents noms, calcule la distance entre les deux pages de wikipedia associées. La distance maximale que nous pouvons obtenir est 1.

In [23]:
def distance_names(name1, name2):
    return graphlab.distances.cosine(pages[pages['name'] == name1]['tfidf'][0], pages[pages['name'] == name2]['tfidf'][0])


In [24]:
print distance_names('Barack Obama', 'Bill Clinton')

0.833985493688


In [25]:
print distance_names('Barack Obama', 'Silvio Berlusconi')

0.960869474154


In [26]:
print distance_names('Barack Obama', 'Nicolas Sarkozy')

0.953687259117


In [27]:
print distance_names('Silvio Berlusconi', 'Nicolas Sarkozy')

0.967830338802


In [28]:
print distance_names('Silvio Berlusconi','David Beckham')

0.95816004976


In [29]:
print distance_names('Silvio Berlusconi', 'Britney Spears')

0.971809372456


# Clustering et KMeans


On va maintenant montrer comment utiliser KMeans et en général le clustering pour identifier les pages plus proche d'une page donnée, en utilisant la fonction nearest_neighbors de graphlab.

In [30]:
knn_model = graphlab.nearest_neighbors.create(pages, features = ['tfidf'], label = 'name' )


PROGRESS: Starting brute force nearest neighbors model training.


La fonction knn_name calcule le 5 plus proche personnages, informations obtenues en utilisant le model.

In [31]:
def knn_name(name):
    return knn_model.query(pages[pages['name'] == name])

In [32]:
print knn_name('Barack Obama')

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 24.365ms     |
PROGRESS: | Done         |         | 100         | 657.317ms    |
PROGRESS: +--------------+---------+-------------+--------------+
+-------------+-----------------+----------------+------+
| query_label | reference_label |    distance    | rank |
+-------------+-----------------+----------------+------+
|      0      |   Barack Obama  |      0.0       |  1   |
|      0      |    Joe Biden    | 0.794117647059 |  2   |
|      0      |  Joe Lieberman  | 0.794685990338 |  3   |
|      0      |   Kelly Ayotte  | 0.811989100817 |  4   |
|      0      |   Bill Clinton  | 0.813852813853 |  5   |
+-------------+-----------------+----------------+------+
[5 rows x 4 columns]



In [33]:
print knn_name('Silvio Berlusconi')

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 20.993ms     |
PROGRESS: | Done         |         | 100         | 579.923ms    |
PROGRESS: +--------------+---------+-------------+--------------+
+-------------+--------------------+----------------+------+
| query_label |  reference_label   |    distance    | rank |
+-------------+--------------------+----------------+------+
|      0      | Silvio Berlusconi  |      0.0       |  1   |
|      0      | Paul B%C3%A9renger | 0.830409356725 |  2   |
|      0      |     Mark Rutte     | 0.832116788321 |  3   |
|      0      | Giorgio Napolitano | 0.836419753086 |  4   |
|      0      |    Peter Dunne     | 0.842519685039 |  5   |
+-------------+--------------------+----------------+------+
[5 rows x 4 colum

In [34]:
print knn_name('Jacques Chirac')

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 18.946ms     |
PROGRESS: | Done         |         | 100         | 557.461ms    |
PROGRESS: +--------------+---------+-------------+--------------+
+-------------+---------------------+----------------+------+
| query_label |   reference_label   |    distance    | rank |
+-------------+---------------------+----------------+------+
|      0      |    Jacques Chirac   |      0.0       |  1   |
|      0      |   Alain Jupp%C3%A9  | 0.826254826255 |  2   |
|      0      | Jean-Pierre Soisson | 0.837662337662 |  3   |
|      0      |     Pierre Joxe     | 0.838345864662 |  4   |
|      0      |     Mario Silva     | 0.847272727273 |  5   |
+-------------+---------------------+----------------+------+
[5 rows 

# Remarques finales

En utilisant des techniques d'apprentissage automatique, particulièrement TF-IDF et le clustering, on a trouvé une manière de donner une distance entre certaine pages de wikipedia, dans ce cas les pages concernant des personnages publiques. Pour simplifier la computation on a consideré un sous ensemble des pages de wikipedia, mais la même méthode peut être utilisé pour analyser tous les pages de l'encyclopedie. On pourrait aussi penser d'ajouter une fonction qui associe de manière automatique les pages plus proches à une certaine page visitée.